In [9]:
import json
from openai import AzureOpenAI
import os

endpoint ="https://oai-mtpchatbot-dev-frcent-001.openai.azure.com/"
deployment='oaidepl-mtpchatbot-dev-frcent-001'

client = AzureOpenAI(
    base_url=f"{endpoint}/openai/deployments/{deployment}/",
    api_key="a74bc358eb354a7590d19d43fedc999a",
    api_version="2023-12-01-preview",
    )

rag_client = AzureOpenAI(
    base_url=f"{endpoint}/openai/deployments/{deployment}/extensions",
    api_key="a74bc358eb354a7590d19d43fedc999a",
    api_version="2023-12-01-preview",
    )

def get_trucklive_status(vin: int):
    if vin == 123456789:
        return "acitvated"
    else:
        return "deactivated"

def get_current_contracts(user, password):
    if user == "yannik" and password == 123:
        return "TruckLive, Fleetboard, Uptime"
    else: return "None"


def run_conversation():
    print("Do you want to ask about general information and functionalities of My TruckPoint and related services or do you want me to perform specific tasks? (g for general/ s for specific)?")
    user_choice= ""
    while True:
        user_choice = input()
        if user_choice == "s" or "g":
            break
        else:
            print("Try again")

    messages = []
    messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})

    if user_choice == "s":
        while True:
        # Step 1: send the conversation and available functions to the model
            print("What can I check for you?")
            messages.append({"role": "user", "content": input()})
            tools = [
                {
                    "type": "function",
                    "function": {
                        "name": "get_trucklive_status",
                        "description": "Returns the TruckLive Status of a vehicle given the VIN (Vehicle Identification Number).",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "vin": {
                                    "type": "integer",
                                    "description": "the vehicle identification number",
                                },
                            },
                            "required": ["vin"],
                        },
                    }
                },
                        {
                    "type": "function",
                    "function": {
                        "name": "get_current_contracts",
                        "description": "Return the current contracts the user is subscribed to given the user name",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "user": {
                                    "type": "string",
                                    "description": "the user name",
                                },
                            },
                            "required": ["user"],
                        },
                    }
                },
            ]
            response = client.chat.completions.create(
                model=deployment,
                messages=messages,
                tools=tools,
                tool_choice="auto",  # auto is default, but we'll be explicit
            )
            response_message = response.choices[0].message
            tool_calls = response_message.tool_calls
            # Step 2: check if the model wanted to call a function
            if tool_calls:
                # Step 3: call the function
                # Note: the JSON response may not always be valid; be sure to handle errors
                available_functions = {
                    "get_trucklive_status": get_trucklive_status,
                    "get_current_contracts": get_current_contracts,
                }  # only one function in this example, but you can have multiple
                messages.append(response_message)  # extend conversation with assistant's reply
                # Step 4: send the info for each function call and function response to the model
                for tool_call in tool_calls:
                    function_name = tool_call.function.name
                    function_to_call = available_functions[function_name]
                    function_args = json.loads(tool_call.function.arguments)
                    function_response = function_to_call(**function_args)
                    messages.append(
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": function_response,
                        }
                    )  # extend conversation with function response
                second_response = client.chat.completions.create(
                    model=deployment,
                    messages=messages,
                )  # get a new response from the model where it can see the function response
                return second_response.model_dump_json(indent=2)
                
            else: print(response.model_dump_json(indent=2))
    
    elif user_choice == "g":
        print("What do you want to know?")
        messages.append({"role": "user", "content": input()})
        rag_response = rag_client.chat.completions.create(
            model=deployment,
            messages=messages,
            extra_body={
                "dataSources": [
                    {
                        "type": "AzureCognitiveSearch",
                        "parameters": {
                            "endpoint": os.environ["AZURE_AI_SEARCH_ENDPOINT"],
                            "key": os.environ["AZURE_AI_SEARCH_API_KEY"],
                            "indexName": "mtp-chatbot-hybrid-search-plus-weblate",
                        }
                    }
                ]
            }
        )
        return rag_response.model_dump_json(indent=2)

print(run_conversation())


Do you want to ask about general information and functionalities of My TruckPoint and related services or do you want me to perform specific tasks? (g for general/ s for specific)?
What can I check for you?
{
  "id": "chatcmpl-8zr1iMOfBPN8oWQ7cqVSw8vkJlJ9W",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "To assist you, I need the VIN (Vehicle Identification Number) of your vehicle to check its TruckLive status. Could you please provide the VIN?",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "sev